In [3]:
import pandas as pd
# pip install pyfaidx
from pyfaidx import Fasta
import os
# pip install seaborn
import seaborn as sns
import matplotlib.pyplot as plt

# choice of SAGs
# 1. AG-910-D08
# 2. AG-910-N05 (smallest - quickest)
# 3. AG-910-N11
# 4. AG-910-F10
# 5. AG-910-K03 - really big - 20x of the smallest... 

sag = 'AG-910-N05'

Next, let's check out the quality of these identified vcandidate contigs.  To do so, we first need to extract the viral sequences and write them into a new file, and then we can run them through CheckV.

4. Extract viral contigs from SAG into new Fasta file
5. Run viral contigs through CheckV
6. Curate viral candidate contigs based on checkV results
7. Extract checkV passing viral contigs

First let's set up some variables that we'll be using within this notebook (same as the last notebook).

In [4]:
# setting a working subdirectory for virus finding
sagdir = "{}_vfinding".format(sag)

# setting a location to place SAG contigs
sag_contigs = os.path.join(sagdir, 'final_contigs_{}.fasta'.format(sag))

And load the dataframe we'd put together in notebook 1:

In [5]:
df = pd.read_csv(os.path.join(sagdir, '{}_vfinding_vcandidates_merged_table.csv'.format(sag)))
df

,contig,sag,vs2_pos,vs2_type,dvf_pos,consensus_score
0,AG-910-N05_NODE_1,AG-910-N05,1,full,1,2
1,AG-910-N05_NODE_2,AG-910-N05,1,full,1,2


### Extracting viral sequences

We'll use a package called pyfaidx to grab sequences from our fasta file, and our own function to write them to a new file.

In [6]:
# function to write to a new output file handle:

def write_fa_record(name, seq, oh, line_len=60):
    print(">{}".format(name), file=oh)
    for i in range(0, len(seq), line_len):
        print(seq[i:i+line_len], file=oh)

In [7]:
fa = Fasta(sag_contigs)
vcandidate_fasta = os.path.join(sagdir, '{}_initial_vcandidates.fasta'.format(sag))

with open(vcandidate_fasta, 'w') as oh:
    for i, l in df.iterrows():
        seq = fa[l.contig]
        write_fa_record(l.contig, seq, oh)
        
print("Our fasta file to run through checkv is", vcandidate_fasta)

Our fasta file to run through checkv is AG-910-N05_vfinding/AG-910-N05_initial_vcandidates.fasta


### CheckV of viral candidate sequences

In [8]:
checkv_outdir = os.path.join(sagdir, "checkv")

!mkdir {checkv_outdir}

We'll want to work in the CheckV conda environment to run this software in a terminal, rather than in this notebook. Load this environment in terminal by typing:

```
source activate /mnt/storage/envs/checkv
```

The general command for vibrant is:
```
checkv end_to_end {infa} {outdir} -t {threads} -d {location of checkv database}
```

To run for your SAG, enter the printed output from the below line into your terminal:

In [9]:
print('checkv end_to_end {vcandidate_fasta} {checkv_outdir} -t 2 -d /mnt/storage/reference_dbs/checkv/checkv-db-v1.5'.format(**locals()))

checkv end_to_end AG-910-N05_vfinding/AG-910-N05_initial_vcandidates.fasta AG-910-N05_vfinding/checkv -t 2 -d /mnt/storage/reference_dbs/checkv/checkv-db-v1.5


Let's check out the results!  

In [10]:
!ls {checkv_outdir}

complete_genomes.tsv  contamination.tsv  quality_summary.tsv  viruses.fna
completeness.tsv      proviruses.fna	 tmp


Let's look specifically into the quality summary file:

In [11]:
cvdf = pd.read_csv(os.path.join(checkv_outdir, 'quality_summary.tsv'), sep = "\t")
cvdf

,contig_id,contig_length,provirus,proviral_length,gene_count,viral_genes,host_genes,checkv_quality,miuvig_quality,completeness,completeness_method,contamination,kmer_freq,warnings
0,AG-910-N05_NODE_1,17534,No,NaN,40,12,1,Low-quality,Genome-fragment,39.41,AAI-based (high-confidence),0.0,1.0,NaN
1,AG-910-N05_NODE_2,7372,No,NaN,15,2,0,Low-quality,Genome-fragment,19.73,AAI-based (high-confidence),0.0,1.0,NaN


What's the checkv quality of your SAG's viral candidates?

In [12]:
cvdf['checkv_quality'].value_counts()

checkv_quality
Low-quality    2
Name: count, dtype: int64

A metric I like to look at is the proportion of genes identified as host genes on each contig. Let's add a column that shows this:

In [13]:
cvdf['pct_host_genes'] = round(cvdf['host_genes'] / cvdf['gene_count'] * 100, 1)

In [14]:
cvdf[['contig_id','pct_host_genes','checkv_quality']]

,contig_id,pct_host_genes,checkv_quality
0,AG-910-N05_NODE_1,2.5,Low-quality
1,AG-910-N05_NODE_2,0.0,Low-quality


We'll select contigs to move forward with based on the following criteria:

```
Keep IF:
checkv_quality is NOT Not-determined
OR IF:
pct_host_genes < 50%
```

In [15]:
cvdf_keeps = cvdf[(cvdf['checkv_quality'] != 'Not-determined') | (cvdf['pct_host_genes'] < 50)]



In [16]:
cvdf_keeps

,contig_id,contig_length,provirus,proviral_length,gene_count,viral_genes,host_genes,checkv_quality,miuvig_quality,completeness,completeness_method,contamination,kmer_freq,warnings,pct_host_genes
0,AG-910-N05_NODE_1,17534,No,NaN,40,12,1,Low-quality,Genome-fragment,39.41,AAI-based (high-confidence),0.0,1.0,NaN,2.5
1,AG-910-N05_NODE_2,7372,No,NaN,15,2,0,Low-quality,Genome-fragment,19.73,AAI-based (high-confidence),0.0,1.0,NaN,0.0


Finally, let's save this table for the next step.

In [17]:
cvdf_keeps.to_csv(os.path.join(sagdir, '{}_post_qc_vcandidates_checkv_quality_table.csv'.format(sag)), index = False)

OK, finally, let's extract the passing checkV contigs into a new fasta file for input for the annotation step.

This time, we'll extract contigs from the checkV output. We will do it this way because checkV separates prophages from host sequences, so if CheckV determined any of our sequences were prophage, we will only look at the phage portion of the sequence for our next step.

In [18]:
vfa = Fasta(os.path.join(checkv_outdir, 'viruses.fna'))
              
vcandidate_fasta = os.path.join(sagdir, '{}_cvpassing_vcandidates.fasta'.format(sag))

with open(vcandidate_fasta, 'w') as oh:
    for i, l in cvdf.iterrows():
        
        if l['provirus'] == 'Yes':
            profa = Fasta(os.path.join(checkv_outdir, 'proviruses.fna'))
            seq = profa[l['contig_id']]
        else:
            seq = vfa[l['contig_id']]
    
        write_fa_record(l['contig_id'], seq, oh)
        
print("Our fasta file to run through DRAM-v is", vcandidate_fasta)

Our fasta file to run through DRAM-v is AG-910-N05_vfinding/AG-910-N05_cvpassing_vcandidates.fasta


Questions:

How much did the CheckV filter affect the final contig counts?

How are you feeling about your SAG at this point?  Do you think it contains a virus?

Do you agree with the decisions in this notebook for filtering out viral contigs?